In [1]:
import string
import json 
import pickle as pkl
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pyspark as ps
from collections import Counter
import numpy as np

In [2]:
sc = ps.SparkContext()

In [3]:
test_strings = ['the quick brown fox jumps over the brown fence.',
              'the boy paints a tall fence brown!',
              'basketball players are tall.',
              'quick basketball players jump high']

In [4]:
import nltk, string

def tokenize(text):
    tokens = [] 
    
    for word in nltk.word_tokenize(text):
        if word \
            not in nltk.corpus.stopwords.words('english') \
            and word not in string.punctuation \
            and word != '``':    
                tokens.append(word)
    
    return tokens

In [5]:
test_tokens = sc.parallelize(test_strings).map(tokenize)
test_tokens.collect()

[['quick', 'brown', 'fox', 'jumps', 'brown', 'fence'],
 ['boy', 'paints', 'tall', 'fence', 'brown'],
 ['basketball', 'players', 'tall'],
 ['quick', 'basketball', 'players', 'jump', 'high']]

In [6]:
vocab = test_tokens.flatMap(lambda words: words).distinct()
vocab.collect()

['quick',
 'jump',
 'high',
 'brown',
 'players',
 'tall',
 'paints',
 'boy',
 'jumps',
 'basketball',
 'fox',
 'fence']

In [7]:
vocab.count()

12

In [8]:
from collections import Counter
import numpy as np

broadcastVocab = sc.broadcast(vocab.collect())

def bow_vectorize(tokens):
    word_counts = Counter(tokens)
    vector = [word_counts[v] if v in word_counts else 0 for v in broadcastVocab.value]
    return np.array(vector)

In [9]:
test_tokens.map(bow_vectorize).collect()

[array([1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 1, 1]),
 array([0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1]),
 array([0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0]),
 array([1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0])]

In [10]:
test_tokens.collect()

[['quick', 'brown', 'fox', 'jumps', 'brown', 'fence'],
 ['boy', 'paints', 'tall', 'fence', 'brown'],
 ['basketball', 'players', 'tall'],
 ['quick', 'basketball', 'players', 'jump', 'high']]

In [11]:
broadcastVocab.value

['quick',
 'jump',
 'high',
 'brown',
 'players',
 'tall',
 'paints',
 'boy',
 'jumps',
 'basketball',
 'fox',
 'fence']

## TF-IDF

In [12]:
term_freq = test_tokens.map(lambda terms: Counter(terms))

In [13]:
doc_freq = term_freq.flatMap(lambda counts: counts.keys()).map(lambda keys: (keys, 1)).reduceByKey(lambda a, b: a + b)

In [14]:
total_docs = term_freq.count()

In [15]:
import math

idf = map(lambda tup: (tup[0], math.log(float(total_docs)/ (1 + tup[1]))), doc_freq.collect())

In [16]:
broadcast_idf = sc.broadcast(idf)

def tfidf_vectorize(tokens):
    word_counts = Counter(tokens)
    doc_length = sum(word_counts.values())
    
    vector = [ word_counts.get(word[0], 0) * word[1] / float(doc_length) for word in broadcast_idf.value]
    return np.array(vector)

In [17]:
test_tokens.map(tfidf_vectorize).collect()

[array([ 0.04794701,  0.        ,  0.        ,  0.09589402,  0.        ,
         0.        ,  0.        ,  0.        ,  0.11552453,  0.        ,
         0.11552453,  0.04794701]),
 array([ 0.        ,  0.        ,  0.        ,  0.05753641,  0.05753641,
         0.13862944,  0.        ,  0.13862944,  0.        ,  0.        ,
         0.        ,  0.05753641]),
 array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.09589402,
         0.        ,  0.09589402,  0.        ,  0.        ,  0.09589402,
         0.        ,  0.        ]),
 array([ 0.05753641,  0.13862944,  0.13862944,  0.        ,  0.        ,
         0.        ,  0.05753641,  0.        ,  0.        ,  0.05753641,
         0.        ,  0.        ])]

In [18]:
test_tokens.map(bow_vectorize).collect()

[array([1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 1, 1]),
 array([0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1]),
 array([0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0]),
 array([1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0])]